In [1]:
from CC import CC
from DnsVhost import DnsVhost
from MariaDB import MariaDB
from WP import WP
#from Partition import Partition
from SMB import SMB

## 객체 ###
cc =  CC("172.16.16.25", "root", "asd123!@")
cc.connect()
dv = DnsVhost() 
mariadb = MariaDB()
wp = WP()
#part = Partition()
smb = SMB("Rocky")

## 전역변수 ##
ZONE_FILE = ""
IPADDR = ""


###################################################
#              HTTP,DNS -DnsVhost.py              #
###################################################
# 나중에 수정할 필요 있음. DB에 맞춰..
#---------------------HTTP-----------------------#
## 1. http 설치, vhost 수정, 재시작 , DNS설치 및 설정까지 : 밑의 mod_http_R 함수 사용
def install_and_setting_http() :
    cc.run(dv.install_http())
    mod_http_and_dns()

## 2. vhost 수정, 재시작
def mod_http_and_dns():
    cc.run(dv.set_vhostconf())
    set_httpd_and_dns()
    cc.run(dv.restart_http())

## 3. vhost 수정, 만약 zone파일이 존재하면 vhost의 내용을 반영하고 3차도메인을 수정합니다.
def set_httpd_and_dns() :
    vhost_conf = dv.getter_VHOST_CONF()
    print(f"현재 당신의 vhost.conf 파일은 {vhost_conf}입니다.")
    while(True) :
        cc.run(dv.show_vhostconf())
        num = int(input("(1) vhost 추가하기, (0) vhost 파일 수정 종료"))
        if(num == 0) :
            print("vhost 파일 수정을 종료합니다")
            break;
        cc.run(dv.mod_vhostconf())
    cc.run(dv.restart_http())

    print("httpd와 관련한 vhost내용은 성공적으로 반영되었습니다. 이에따른 DNS 내용도 수정하겠습니다")

    ## DNS 내용들임
    install_and_setting_dns()
    #cc.dv.set_dns_Third_domain()

        
#---------------------DNS-----------------------#
## 1. DNS 설치 및 ZONE 파일 수정, vhost에 따른 3차도메인 자동 업데이트
def install_and_setting_dns():
    cc.run(dv.install_named())
    cc.run(dv.set_named_conf())
    cc.run(dv.set_named_rfc())
    cc.run(dv.mk_zone_file())
    
    cc.run(dv.set_zone_file())
    cc.run(dv.set_dns_Third_domain())
    cc.run(dv.start_dns())

#---------------------실행부-----------------------#
#install_and_setting_http()



###################################################
#            MariaDB,PHP -MariaDB.py              #
###################################################
## 1. mariaDB와 php 둘 다 설치함
def install_mariaDB_and_php():
    cc.run(mariadb.install_packages())
    cc.run(mariadb.install_apache_and_php())
    cc.run(mariadb.install_mariaDB())
    cc.run(mariadb.init_mariaDB())
    cc.run(mariadb.install_php_my_admin())
    cc.run(mariadb.set_phpMyadmin_conf())
#---------------------실행부-----------------------#
#install_mariaDB_and_php()


###################################################
#            Partition -Partition.py              #
###################################################
# def mk_partition():
#     cc.run("lsblk")
#     cc.run(part.make_partition())
#     cc.run(part.format_partition())
#     cc.run(part.mount())
#     cc.run(part.permanent_mount_config_remote())


#---------------------실행부-----------------------#
#mk_partition()

###################################################
#                    SMB -SMB.py                  #
###################################################

def install_smb():
    cc.run(smb.smb_install())
    cc.run(smb.smb_intsll_status_check())   
    choose = input("smb 서버와 클라이언트 중 선택하세요.(S/C) ex)S : ")
    if(choose == "S" ) :
        server_smb()
    if(choose == "C" ) :
        client_smb()

def server_smb():
    cc.run(smb.smb_share_folder_make())
    cc.run(smb.smb_mk_smb_id())
    cc.run(smb.smb_set_smb_id_passwd())
    cc.run(smb.smb_id_check())
    cc.run(smb.smb_share_folder_authority())
    cc.run(smb.smb_mk_back())
    cc.run(smb.smb_set())
    cc.run(smb.smb_server_start())
    
def client_smb() :
    cc.run(smb.smb_client_login())
    cc.run(smb.smb_client_mount())
#---------------------실행부-----------------------#
#install_smb()

    
###################################################
#                 WP,PHPMYSQL -WP.py              #
###################################################
## 1. 로컬) WP 설치 및 wp-config.php 파일 수정까지
def install_wp():
    wp.install_prev_packages()
    wp.install_and_move_dir()
    wp.set_wp_config_p()

실행
DnsVhost init start
MariaDB init start
WP init 시작
smb 패키지를 설치합니다.
Last metadata expiration check: 0:05:46 ago on Tue Dec  9 12:00:12 2025.
Package samba-4.22.4-6.el9.x86_64 is already installed.
Package samba-client-4.22.4-6.el9.x86_64 is already installed.
Package cifs-utils-7.2-1.el9.x86_64 is already installed.
Dependencies resolved.
Nothing to do.
Complete!

설치된 smb 패키지를 확인합니다.
samba-common-4.22.4-6.el9.noarch
samba-common-libs-4.22.4-6.el9.x86_64
samba-client-libs-4.22.4-6.el9.x86_64
samba-libs-4.22.4-6.el9.x86_64
samba-dcerpc-4.22.4-6.el9.x86_64
samba-ldb-ldap-modules-4.22.4-6.el9.x86_64
samba-common-tools-4.22.4-6.el9.x86_64
samba-client-4.22.4-6.el9.x86_64
samba-4.22.4-6.el9.x86_64



smb 서버와 클라이언트 중 선택하세요.(S/C) ex)S :  C
접속할 smb 서버의 ip주소를 입력하세요. ex)172.16.12.100 or 도메인: 172.16.16.122
본인(클라이언트) 서버에 연결할 디렉토리를 지정하세요. ex) /mnt/win_share :  /mnt/win_share
smb id를 입력하세요. ex)smb :  smb
smb password를 입력하세요. ex)1234 :  1234


SMB 서버에 로그인하겠습니다
spawn smbclient -U smb -L 172.16.16.122
Password for [SAMBA\smb]:

	Sharename       Type      Comment
	---------       ----      -------
	print$          Disk      Printer Drivers
	share           Disk      smb's SMB
	IPC$            IPC       IPC Service (Samba 4.22.4)
	smb             Disk      Home Directories
SMB1 disabled -- no workgroup available



서버에 지정된 공유 폴더의 이름을 입력하세요. ex)share :  share
/etc/fstab에 부팅 시 자동마운트 설정을 하시겠습니까? n 입력시 수동마운트 적용 ex) y/n :  n


Host "172.16.16.122" resolved to the following IP addresses: 172.16.16.122
mount.cifs kernel mount options: ip=172.16.16.122,unc=\\172.16.16.122\share,vers=3.0,user=smb,pass=********

